In [1]:
from PYME.LMVis import VisGUI

%gui wx

Trying to load 3rd party recipe module cc_drift_cor.plugins.recipes.chung_cc
Loaded 3rd party recipe module cc_drift_cor.plugins.recipes.chung_cc
Trying to load 3rd party recipe module cc_drift_cor.plugins.recipes.processing
Loaded 3rd party recipe module cc_drift_cor.plugins.recipes.processing
Trying to load 3rd party recipe module ch_shrinkwrap.recipe_modules.surface_fitting
Loaded 3rd party recipe module ch_shrinkwrap.recipe_modules.surface_fitting
Trying to load 3rd party recipe module nep_fitting.recipe_modules.nep_fits
Loaded 3rd party recipe module nep_fitting.recipe_modules.nep_fits
Trying to load 3rd party recipe module ch_shrinkwrap.recipe_modules.surface_feature_extraction
Loaded 3rd party recipe module ch_shrinkwrap.recipe_modules.surface_feature_extraction
Trying to load 3rd party recipe module cc_drift_cor.plugins.recipes.io
Loaded 3rd party recipe module cc_drift_cor.plugins.recipes.io
Trying to load 3rd party recipe module pyme_picasso.recipe_modules.conversion
Loaded 3

INFO:PYME.misc.check_for_updates:Checking for updates ...
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): www.python-microscopy.org:80
DEBUG:urllib3.connectionpool:http://www.python-microscopy.org:80 "GET /current_version.json HTTP/1.1" 200 22
INFO:PYME.misc.check_for_updates:A new version of PYME is available
You have version 22.05.25.post0.dev, the current version is 22.10.17
INFO:PYME.misc.check_for_updates:Detected a .git folder, assuming a development install
INFO:PYME.misc.check_for_updates:You have a development install, to update, execute the following in the repository root folder:
    
    git pull origin master
    python setup.py build_ext -i
    
NB: because this is a developent install, this will update to the bleeding edge (i.e. past the last official release)


Detected a .git folder, assuming a development install
c:\users\laf62\code\python-microscopy


In [2]:
pymevis = VisGUI.ipython_pymevisualize()
pipeline = pymevis.pipeline

DEBUG:PYME.LMVis.visCore:Creating VisGUI menu bar
DEBUG:PYME.LMVis.Extras:Initializing QPobjectSegment plugin


quaternion module not found, disabling custom clip plane orientations
quaternion module not found, disabling custom clip plane orientations
New Canvas


DEBUG:PYME.LMVis.Extras:Initializing animation plugin
DEBUG:PYME.LMVis.Extras:Initializing clusterAnalysis plugin
DEBUG:PYME.LMVis.Extras:Initializing dockedPanel plugin
DEBUG:PYME.LMVis.Extras:Initializing extra_layers plugin
DEBUG:PYME.LMVis.Extras:Initializing fiducial plugin
DEBUG:PYME.LMVis.Extras:Initializing flagZMotionArtifacts plugin
DEBUG:root:Adding menu items for event filters
DEBUG:PYME.LMVis.Extras:Initializing fusionUtils plugin
DEBUG:PYME.LMVis.Extras:Initializing multiColourAnalysis plugin
DEBUG:PYME.LMVis.Extras:Initializing multiviewMapping plugin
DEBUG:root:Adding menu items for multi-view manipulation
DEBUG:PYME.LMVis.Extras:Initializing objectMeasurements plugin
DEBUG:PYME.LMVis.Extras:Initializing particleTracking plugin
DEBUG:PYME.LMVis.Extras:Initializing perFrameVariable plugin
DEBUG:PYME.LMVis.Extras:Initializing photophysics plugin
DEBUG:PYME.LMVis.Extras:Initializing pointSetGeneration plugin
DEBUG:PYME.LMVis.Extras:Initializing pointwiseColoc plugin
DEBUG:

Saved Views are deprecated -  use the ones with animation instead and remove me!
Creating fold panel
GenPanels
Creating datasource panel
RecipeView._layout
Input datasources:  []
Image list size: 1
Creating layers GUI
height:  0
NB best size: wx.Size(0, 0)
RecipeView._layout


RuntimeError: wrapped C/C++ object of type foldPanel has been deleted

Input datasources:  []
recipe.execute()
RecipeView._layout


RuntimeError: wrapped C/C++ object of type foldPanel has been deleted

Input datasources:  []
0
OpenGL - Version: b'4.6.0 NVIDIA 516.94'
Shader - Version: b'4.60 NVIDIA'
GL.GL_MAX_SAMPLES: 32, GL.GL_SAMPLES: 4


DEBUG:PYME.LMVis.shader_programs.ShaderProgramFactory:New shader program created: <class 'PYME.LMVis.shader_programs.DefaultShaderProgram.DefaultShaderProgram'>


Ev Idle
('points', 't')
refreshed


In [3]:
import numpy as np
from pathlib import Path
import os
import matplotlib.pyplot as plt
from PYME.Analysis.points import cluster_morphology

CB_color_cycle = ['#377eb8', '#ff7f00', '#4daf4a',
                  '#f781bf', '#a65628', '#984ea3',
                  '#999999', '#e41a1c', '#dede00']

#Parameters to change

# filepath where graphs will be saved
savedir = 'K:\\4Pi_data\\Oligomer_analysis\\Oligomer_analysis_V2\\updated_analysis_20230111\\analysis'
# filepath where data to be analyzed is found
save_loc = "K:\\4Pi_data\\Oligomer_analysis\\Oligomer_analysis_V2\\updated_analysis_20230111"
# The frequency of different oligomer sizes taken from the counting data
c_info = "K:\\4Pi_data\\Oligomer_analysis\\Oligomer_analysis_V2\\updated_analysis_20230111\\count_info_V1.npy"
# location of observed, circular-shaped, and linear-shaped clusters
data = save_loc + '\\cluster_output_V1-adj.npy'
circ = save_loc + '\\circular_sims_V1-adj.npy'
line = save_loc + '\\linear_sims_V1-adj.npy'
adj = save_loc + '\\adj_sims_V1-adj.npy'
# data = save_loc + '\\cluster_output_test_data.npy'
# circ = save_loc + '\\circular_sims_test_data.npy'
# line = save_loc + '\\linear_sims_test_data.npy'
ver = 'V3'
min_size = 10  # Threshold for the minimum counts a cluster may contain
max_size = 50  # Threshold for the maximum coutns a cluster may contain
sim_num = 50   # number of simulated random orientations for each cluster. Must be the same as what
               # was used when creating the simulations
m_dist = 9  # distance between monomers in nanometers
lpm = 3.23     # factor that changes # of localizations per cluster distribution into one that looks like
               # the number of molecules per cluster in the counting data
lp = 5.65      # this is the average localization precision from the three Rtn4 experiments
# Bins for 2D Histogram
nbins = [np.linspace(-90,90,19),np.linspace(0,180,19)]
cent_bins = 24 #number of bins for center polar histogram

# Import data
real = np.load(data, allow_pickle=True)
#real = real.item()
circ_sim = np.load(circ, allow_pickle=True)
#circ_sim = circ_sim.item()
line_sim = np.load(line, allow_pickle=True)
#line_sim = line_sim.item()
adj_sim = np.load(adj, allow_pickle=True)

count_info = np.load(c_info, allow_pickle=True)
count_info = count_info.item()

In [11]:
# Plot alpha(aka psi) angles
"""
==========================
Plot alpha(aka psi) angles
==========================

"""
for a in range(len(real)):
    if a == 0:
        alpha = real[a]['alpha']
        sim_alpha = real[a]['sim_alpha']
        count = real[a]['count']
        # center_angles = real[a]['center_angles']
        # center_sep.append(np.asarray(real[a]['center_angles']))
    else:
        np.concatenate((alpha,real[a]['alpha']))
        np.concatenate((sim_alpha, real[a]['sim_alpha']))
        np.concatenate((count, real[a]['count']))
        # np.concatenate((center_angles, real[a]['center_angles']))
        # center_sep.append(np.asarray(real[a]['center_angles']))

alpha_deg = alpha * (180/np.pi)
sim_alpha_deg = sim_alpha * (180/np.pi)
xticks = [-90,-45,0,45,90]

min_size_filt = count > min_size
max_size_filt = count < max_size
size_filt = min_size_filt * max_size_filt

fig = plt.figure(figsize=[2.5,2.5])
plt.hist(alpha_deg[size_filt],bins=18,density=True, alpha=0.75, range=(-90,90), 
         color=CB_color_cycle[1], edgecolor=CB_color_cycle[1], label='observed',
        histtype='step',linewidth=2)
plt.hist(sim_alpha_deg,bins=18,density=True,alpha=0.75, range=(-90,90),
         color=CB_color_cycle[6], edgecolor=CB_color_cycle[6], label='simulated',
        histtype='step',linewidth=2)
plt.xlabel(u'\u03A8 (degrees)')
plt.ylabel('Normalized frequency')
plt.xticks(xticks)
plt.legend()
plt.show()
psi_save = savedir + '\\psi_histogram_' + ver + '.png'
plt.savefig(psi_save, dpi=600, facecolor='w', edgecolor='w',
        orientation='portrait', papertype=None, format=None,
        transparent=False, bbox_inches='tight', pad_inches=0.1, metadata=None)

# small clusters only
small_clusters = count < min_size
fig = plt.figure(figsize=[2.5,2.5])
plt.hist(alpha_deg[small_clusters], bins=18, density=True, alpha=0.75,
         range=(-90,90), color=CB_color_cycle[0], edgecolor=CB_color_cycle[0],label='observed',
        histtype='step',linewidth=2)
plt.hist(sim_alpha_deg,bins=18, density=True, alpha=0.75,
         range=(-90,90), color=CB_color_cycle[6], edgecolor=CB_color_cycle[6],label='simulated',
        histtype='step',linewidth=2)
plt.xlabel(u'\u03A8 (degrees) of small clusters (counts < ' + str(min_size) + ')')
plt.ylabel('Normalized frequency')
plt.xticks(xticks)
plt.legend(loc='upper center')
plt.show()
alpha_small_save = savedir + '\\alpha_small_cluster_angles' + ver + '.png'
plt.savefig(alpha_small_save, dpi=600, facecolor='w', edgecolor='w',
        orientation='portrait', papertype=None, format=None,
        transparent=False, bbox_inches='tight', pad_inches=0.1, metadata=None)

# medium clusters only
fig = plt.figure(figsize=[2.5,2.5])
plt.hist(alpha_deg[size_filt],bins=18, density=True, alpha=0.75,
         range=(-90,90), color=CB_color_cycle[1], edgecolor=CB_color_cycle[1],label='observed',
        histtype='step',linewidth=2)
plt.hist(sim_alpha_deg,bins=18, density=True, alpha=0.75,
         range=(-90,90), color=CB_color_cycle[6], edgecolor=CB_color_cycle[6],label='simualated',
        histtype='step',linewidth=2)
plt.xlabel(u'\u03A8 (degrees) of medium clusters (' + str(min_size) + ' < counts < ' + str(max_size) + ')')
plt.ylabel('Normalized frequency')
plt.xticks(xticks)
plt.legend(loc='upper center')
plt.show()
alpha_medium_save = savedir + '\\alpha_medium_cluster_angles' + ver + '.png'
plt.savefig(alpha_medium_save, dpi=600, facecolor='w', edgecolor='w',
        orientation='portrait', papertype=None, format=None,
        transparent=False, bbox_inches='tight', pad_inches=0.1, metadata=None)

# large clustrs only
large_clusters = count > max_size
fig = plt.figure(figsize=[2.5,2.5])
plt.hist(alpha_deg[large_clusters], bins=18, density=True, alpha=0.75,
         range=(-90,90), color=CB_color_cycle[2], edgecolor=CB_color_cycle[2],label='observed',
        histtype='step',linewidth=2)
plt.hist(sim_alpha_deg,bins=18, density=True,alpha=0.75,
         range=(-90,90), color=CB_color_cycle[6], edgecolor=CB_color_cycle[6],label='simulated',
        histtype='step',linewidth=2)
plt.xlabel(u'\u03A8 (degrees) of large clusters (counts > '+ str(max_size) + ')')
plt.ylabel('Normalized frequency')
plt.xticks(xticks)
plt.legend(loc='upper center')
plt.show()
alpha_large_save = savedir + '\\alpha_large_cluster_angles' + ver + '.png'
plt.savefig(alpha_large_save, dpi=600, facecolor='w', edgecolor='w',
        orientation='portrait', papertype=None, format=None,
        transparent=False, bbox_inches='tight', pad_inches=0.1, metadata=None)

# Small, medium, and large clusters together
fig = plt.figure(figsize=[2.5,2.5])
plt.hist(alpha_deg[small_clusters], bins=18, density=False, alpha=0.75,
         range=(-90,90), color=CB_color_cycle[0], edgecolor=CB_color_cycle[0],
         label='small clusters (counts > '+ str(min_size) + ')',
        histtype='step',linewidth=2)
plt.hist(alpha_deg[size_filt], bins=18, density=False, alpha=0.75,
         range=(-90,90), color=CB_color_cycle[1], edgecolor=CB_color_cycle[1],
         label='medium clusters (' + str(min_size) + ' < counts < ' + str(max_size) + ')',
        histtype='step',linewidth=2)
plt.hist(alpha_deg[large_clusters], bins=18, density=False, alpha=0.75,
         range=(-90,90), color=CB_color_cycle[2], edgecolor=CB_color_cycle[2],
         label='large clusters (counts > '+ str(max_size) + ')',
        histtype='step',linewidth=2)
#plt.legend(loc='upper center')
plt.xlabel(u'\u03A8 (degrees)')
plt.ylabel('Normalized frequency')
plt.xticks(xticks)
plt.show()
alpha_all_size_save = savedir + '\\alpha_all_size_cluster_angles' + ver + '.png'
plt.savefig(alpha_all_size_save, dpi=600, facecolor='w', edgecolor='w',
        orientation='portrait', papertype=None, format=None,
        transparent=False, bbox_inches='tight', pad_inches=0.1, metadata=None)

C:\Users\laf62\.conda\envs\pyme\lib\site-packages\ipykernel_launcher.py:30: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
C:\Users\laf62\.conda\envs\pyme\lib\site-packages\ipykernel_launcher.py:49: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
C:\Users\laf62\.conda\envs\pyme\lib\site-packages\ipykernel_launcher.py:67: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the r

In [12]:
"""
====================================================
Histogram on polar axis of cluster center phi angles
====================================================

"""
import numpy as np
import matplotlib.pyplot as plt
from numpy import pi

center_sep = []
for a in range(len(real)):
    if a == 0:
        center_angles = real[a]['center_angles']
        center_sep.append(np.asarray(real[a]['center_angles']))
    else:
        np.concatenate((center_angles, real[a]['center_angles']))
        center_sep.append(np.asarray(real[a]['center_angles']))

n_it = np.zeros(len(center_sep),dtype=list)
bins_it = np.zeros(len(center_sep),dtype=list)
patches_it = np.zeros(len(center_sep),dtype=list)
for b in range(len(center_sep)):
    n, bins, patches = plt.hist(x=center_sep[b],bins=cent_bins,
                                alpha=0.3, rwidth=0.85, density=True, 
                                color='blue', range=(-pi,pi))
    n_it[b] = n
    bins_it[b] = bins
    patches_it[b] = patches
    del n, bins, patches
center_avg = np.average(n_it,axis=0)
center_std = np.std (n_it,axis=0)
b_width = bins_it[0][1]-bins_it[0][0]

x = np.zeros(len(bins_it[0])-1)
for d in range(len(bins_it[0])-1):
    x[d] = (bins_it[0][d]+bins_it[0][d+1])/2


plt.rcParams.update({'font.size': 9})
fig = plt.figure(figsize=[2.5,2.5])
ax = plt.subplot(111, projection='polar')
hist = ax.bar(x, center_avg, alpha=0.75, width=b_width, 
             color='tab:blue', label='Rtn4 cluster centers',
             edgecolor='k', linewidth=0.5, yerr=center_std, capsize=0, 
             ecolor='tab:blue')
# hist = ax.hist(x=center_angles, bins=32, alpha=1, rwidth=1, density=True, 
#                color=CB_color_cycle[0], edgecolor='k', linewidth=0.5, label='Rtn4 Cluster Centers')
ax.set_rlabel_position(180)  # Move radial labels away from plotted line
ax.set_rticks([0.1,0.2,0.3])
ax.set_theta_zero_location("N")
plt.xlabel(u'\u03C6 (degrees)')
polar_cent_save = savedir + '\\polar_angles_of_cluster_centers_all' + ver + '.png'
plt.savefig(polar_cent_save, dpi=600, facecolor='w', edgecolor='w',
        orientation='portrait', papertype=None, format=None,
        transparent=False, bbox_inches='tight', pad_inches=0.1, metadata=None)

C:\Users\laf62\.conda\envs\pyme\lib\site-packages\ipykernel_launcher.py:41: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


Done: <matplotlib.backends.backend_agg.RendererAgg object at 0x0000015747F27848>

DEBUG:matplotlib.axes._base:not adjusting title pos because a title was already placed manually: 1.050000
DEBUG:matplotlib.axes._base:title position was updated manually, not adjusting
DEBUG:matplotlib.axes._base:title position was updated manually, not adjusting
DEBUG:matplotlib.axes._base:title position was updated manually, not adjusting
DEBUG:matplotlib.axes._base:title position was updated manually, not adjusting
DEBUG:matplotlib.axes._base:title position was updated manually, not adjusting
DEBUG:matplotlib.axes._base:title position was updated manually, not adjusting


In [13]:
"""
==============================================================================
Plot 2D histogram between alpha(psi) angles and phi angles for cluster centers
==============================================================================

"""
from numpy import pi
# convert to degrees
all_center_deg = np.asarray(center_angles) * (180/pi)
xticks = [-90,-45,0,45,90]
yticks = [0,45,90,135,180]
# Plot a 2D histogram of the angles between the cluster major axis and the tubule axis (x) and tubule z-axis(y)
plt.rcParams.update({'font.size': 9})
fig = plt.figure(figsize=[2.5,2.5])
abs_center_deg = np.absolute(all_center_deg)
h, xedges, yedges, image = plt.hist2d(alpha_deg[size_filt], abs_center_deg[size_filt], bins=nbins, cmap=plt.cm.inferno)#, density=True)
# plt.ylim([-180,180])
plt.xlim([-90,90])
plt.xticks(xticks)
plt.yticks(yticks)
plt.xlabel(u'\u03C8 (degrees)')
plt.ylabel(u'\u03C6 (degrees)')
hist_2d_save = savedir + '\\angles_between_axes_2D_histogram' + ver + '.png'
plt.savefig(hist_2d_save, dpi=600, facecolor='w', edgecolor='w',
        orientation='portrait', papertype=None, format=None,
        transparent=False, bbox_inches='tight', pad_inches=0.1, metadata=None)

C:\Users\laf62\.conda\envs\pyme\lib\site-packages\ipykernel_launcher.py:14: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  


Done: <matplotlib.backends.backend_agg.RendererAgg object at 0x0000015749B1F908>

In [16]:
"""
========================================================
Plot anisotropies and Simulate monomers within a cluster
========================================================

Simulation details:
- Used the 3.23 as the poisson lambda to determine number of localizations per molecule
    - This is the factor that seems to roughly convert 4Pi data from localizations per cluster to molecules per cluster at least judging from their distributions

- Distance between molecules is 4.5 nm
    - Theoretically derived by plugging in the sequence of Rtn4b to the calculator at:
        http://biotools.nubic.northwestern.edu/proteincalc.html
        - It gives the volume and if its assumed the protein is spherical, the radius can be
            determined
            
- Localization precision: 5.26 nm
    - The average localization precision from the real data
    
- The frequency of different oligomer sizes was taken from the counting data 'c_info'

"""
clusters = []
rng = np.random.default_rng()

min_oli = int(np.rint(min_size/lpm))
max_oli = int(np.rint(max_size/lpm))
frequency = count_info['frequency'][min_oli:max_oli]
oli_size = count_info['oli-size'][min_oli:max_oli]

for a in range(len(oli_size)):                                  # for each oligomer size
    for b in range(int(frequency[a])):                          # for the number of times that oligomer size occurs
        pois = rng.poisson(lpm,int(oli_size[a]))                     # determine number of localizations each molecule has
        total = sum(pois)                                       # find total number of localizations for this cluster
        count = 0
        points = np.zeros((total,3))
        for c in range(int(oli_size[a])):                            # for each of molecule in this cluster
            for d in range(pois[c]):                            # for each localization in each molecule in this cluster
                points[count] = np.array([m_dist*c,0,0])
                points[count][0] = points[count][0] + rng.normal(loc=0,scale=lp,size=1)
                points[count][1] = points[count][1] + rng.normal(loc=0,scale=lp,size=1)
                points[count][2] = points[count][2] + rng.normal(loc=0,scale=lp,size=1)
                count = count + 1
        clusters.append(points)

# Unpack wanted measurements
for z in range(len(real)):
    if z == 0:
        real_ani = real[z]['anisotropy']
        circ_ani = circ_sim[z]['anisotropy']
        line_ani = line_sim[z]['anisotropy']
        adj_ani = adj_sim[z]['anisotropy']
        ccounts = real[z]['count']
    else:
        np.concatenate((real_ani, real[z]['anisotropy']))
        np.concatenate((circ_ani, circ_sim[z]['anisotropy']))
        np.concatenate((line_ani, line_sim[z]['anisotropy']))
        np.concatenate((adj_ani, adj_sim[z]['anisotropy']))
        np.concatenate((ccounts, real[z]['count']))

min_filt = ccounts > min_size
max_filt = ccounts < max_size
size_filt = min_filt * max_filt
real_ani_filt = real_ani[size_filt]
circ_ani_filt = circ_ani[size_filt]
line_ani_filt = line_ani[size_filt]
adj_ani_filt = adj_ani[size_filt]
min_csize = int(np.rint(min_size/lpm))
max_csize = int(np.rint(max_size/lpm))
# Round frequency to nearest integer
frequency = np.rint(count_info['frequency'])[min_csize:max_csize]
# Grab oligomer size bins
oli_size = count_info['oli-size'][min_csize:max_csize]

cluster_anisotropy = np.zeros(len(clusters))
for e in range(len(clusters)):
    if len(clusters[e]) > 2:
        cluster_meas = cluster_morphology.measure_3d(x=clusters[e][:,0],y=clusters[e][:,1],
                                                  z=clusters[e][:,2])
        cluster_anisotropy[e] = cluster_meas['anisotropy'][0]
    else:
        continue

CB_color_cycle = ['#377eb8', '#ff7f00', '#4daf4a',
                  '#f781bf', '#a65628', '#984ea3',
                  '#999999', '#e41a1c', '#dede00']

fig = plt.figure(figsize=[3,2])
plt.hist(circ_ani_filt,bins=20, range=[0,1], alpha=0.5,label='Circular', 
         density=True, color='gold', edgecolor='gold',
        histtype='step', linewidth=2)
plt.hist(line_ani_filt,bins=20, range=[0,1], alpha=0.5,label='Linear', 
         density=True, color='lime', edgecolor='lime',
        histtype='step', linewidth=2)
plt.hist(real_ani_filt,bins=20, range=[0,1], alpha=0.5,label='Real', 
         density=True, color=CB_color_cycle[5], edgecolor=CB_color_cycle[5],
        histtype='step', linewidth=2)
plt.legend(loc='upper right', bbox_to_anchor=(1.02, 1.05),frameon=False, markerfirst=False)
plt.xlabel('Anisotropy')
plt.ylabel('Normalized frequency')
plt.show()
ani_save = savedir + '\\Cluster_anisotropy_' + ver + '.png'
plt.savefig(ani_save, dpi=600, facecolor='w', edgecolor='w',
        orientation='portrait', papertype=None, format=None,
        transparent=False, bbox_inches='tight', pad_inches=0, metadata=None)

fig = plt.figure(figsize=[3,2])
plt.hist(cluster_anisotropy, bins=20, range=[0,1], alpha=0.75,label='Sim-mono', 
         density=True, color=CB_color_cycle[6], edgecolor=CB_color_cycle[6],
        histtype='step', linewidth=2)
plt.hist(real_ani_filt,bins=20, range=[0,1], alpha=0.75,label='Real', 
         density=True, color=CB_color_cycle[5], edgecolor=CB_color_cycle[5],
        histtype='step', linewidth=2)
plt.legend(loc='upper right', bbox_to_anchor=(1.02, 1.03),frameon=False, markerfirst=False)
plt.xlabel('Anisotropy')
plt.ylabel('Normalized frequency')
plt.show()
mono_sim_save = savedir + '\\Monomer_sim_anisotropy_' + ver + '-' + str(m_dist) + '.png'
plt.savefig(mono_sim_save, dpi=600, facecolor='w', edgecolor='w',
        orientation='portrait', papertype=None, format=None,
        transparent=False, bbox_inches='tight', pad_inches=0, metadata=None)

fig = plt.figure(figsize=[3,2])
plt.hist(adj_ani_filt,bins=20, range=[0,1], alpha=0.5,label='Linear\n*0.925', 
         density=True, color='gold', edgecolor='gold',
        histtype='step', linewidth=2)
plt.hist(line_ani_filt,bins=20, range=[0,1], alpha=0.5,label='Linear', 
         density=True, color='lime', edgecolor='lime',
        histtype='step', linewidth=2)
plt.hist(real_ani_filt,bins=20, range=[0,1], alpha=0.5,label='Real', 
         density=True, color=CB_color_cycle[5], edgecolor=CB_color_cycle[5],
        histtype='step', linewidth=2)
plt.legend(loc='upper right', bbox_to_anchor=(1.02, 1.03),frameon=False, markerfirst=False)
plt.xlabel('Anisotropy')
plt.ylabel('Normalized frequency')
plt.show()
adj_save = savedir + '\\Cluster-adjusted_anisotropy_' + ver + '.png'
plt.savefig(adj_save, dpi=600, facecolor='w', edgecolor='w',
        orientation='portrait', papertype=None, format=None,
        transparent=False, bbox_inches='tight', pad_inches=0, metadata=None)

C:\Users\laf62\.conda\envs\pyme\lib\site-packages\ipykernel_launcher.py:87: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
DEBUG:matplotlib.axes._base:title position was updated manually, not adjusting
C:\Users\laf62\.conda\envs\pyme\lib\site-packages\ipykernel_launcher.py:106: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
DEBUG:matplotlib.axes._base:title position was updated manually, not adjusting
C:\Users\laf62\.conda\envs\pyme\lib\site-packages\ipykernel_launcher.py:122: RuntimeWarning: More than 20 figures have been opened. Figures created thr